# HMM for decryption


In [1]:
from src.CipherUtils import CipherGenerator
from src.CipherUtils import TextEncoder
from src.ProbabilityMatrix import ProbabilityMatrix
from src.CipherUtils import TextPreProcessor

from src.HMM_utils import map_alphabet_to_numbers, string_to_numbers
from src.HMM_utils import find_mapping, numbers_to_string, invert_mapping
from src.HMM_utils import convert_numbers_to_letters

from src.HMM_functions import Baum_Welch
from src.HMM_functions import compute_f_log, Viterbi_log, reconstruct

import numpy as np

In [2]:
# RICORDATI DI LEVARE LO SPAZIO
hidden_sequence = "people of western europe a landing was made this morning on the coast of france by troops kangaroo jokes quasi vile xilophone zenit "
hidden_sequence = "in germany it seems to be pretty much automatic pretty much all the time in france and spain it all depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners can leave optional tips was while in france where every price on a restaurant menu already includes for service of people said they generally tipped on top even in sweden where tips are generally not expected the figure was but only of italians said they would typically leave a gratuity after a meal out with a rather greater proportion admitting they never left a cent a startling of respondents in the us however and of germans by far the most in europe confessed they would tip sometimes or often even if the service was terrible indicating that for some tipping is not about quality of service at all the findings of the survey will come as a surprise in germany a country that does not generally think of itself as a nation of happy distributors"
# hidden_sequence = " hello banana xilophone key queue zebra cock pussy tits dandy fart though jolly world mum "

preprocessor = TextPreProcessor()
hidden_sequence = preprocessor.lower(text=hidden_sequence)
hidden_sequence = preprocessor.remove_unknown_chars(
    text=hidden_sequence, unknown_chars=preprocessor.unknown_chars(hidden_sequence)
)
hidden_sequence = preprocessor.remove_additional_spaces(text=hidden_sequence)
### WARNING: remember di mettere sempre preprocessor.remove_additional_spaces(text) come ultimo step del preprocessing


cipher_generator = CipherGenerator()
cipher1 = cipher_generator.generate_cipher()
cipher2 = cipher_generator.generate_cipher()
encoder = TextEncoder()
observed_sequence = encoder.encode_text_double_cipher(hidden_sequence, cipher1, cipher2)

print(hidden_sequence)
print(observed_sequence)

in germany it seems to be pretty much automatic pretty much all the time in france and spain it all depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners can leave optiona

In [3]:
# List of text file paths to build our corpus (where we learn the transitions probs)
file_paths = [
    "texts/moby_dick.txt",
    "texts/shakespeare.txt",
    "texts/james-joyce-a-portrait-of-the-artist-as-a-young-man.txt",
    "texts/james-joyce-dubliners.txt",
    "texts/james-joyce-ulysses.txt",
]

texts = []
for file_path in file_paths:
    with open(file_path, "r") as file:
        texts.append(file.read())

corpus = "".join(texts)
alphabet = list("abcdefghijklmnopqrstuvwxyz ")

preprocessor = TextPreProcessor()
corpus = preprocessor.lower(text=corpus)
corpus = preprocessor.remove_unknown_chars(
    text=corpus, unknown_chars=preprocessor.unknown_chars(corpus)
)
corpus = preprocessor.remove_additional_spaces(text=corpus)

# compute probabilities
p = ProbabilityMatrix(corpus)
p.compute_probability_matrix()
p.compute_normalized_matrix()
# p.compute_probability_table()

#### Following line to check consistency of probability_table (the dictionary) and probability_matrix (the matrix)


In [ ]:
p1 = ProbabilityMatrix(corpus)
p1.compute_probability_table()

In [ ]:
dict(sorted(p1.probability_table.items(), key=lambda x: x[0]))

In [ ]:
def compare(prob1, prob2, acc=10):
    # prob1 deve essere matrix
    # prob2 deve essere dict
    i = 0  # number of errors (should be 0 :))
    errors = []
    error_letter = []
    for x in "abcdefghijklmnopqrstuvwxyz ":
        for y in "abcdefghijklmnopqrstuvwxyz ":
            if round(prob1.get_probability_mat(x, y), acc) != round(
                prob2.get_probability(x + y), acc
            ):
                i += 1
                errors.append(
                    prob1.get_probability_mat(x, y) - prob2.get_probability(x + y)
                )
                error_letter.append([x, y])

            elif round(prob1.get_probability_mat(y, x), acc) != round(
                prob2.get_probability(y + x), acc
            ):
                i += 1
                errors.append(
                    prob1.get_probability_mat(x, y) - prob2.get_probability(x + y)
                )
                error_letter.append([y, x])

    return i, errors, error_letter


i, errors, error_letter = compare(p, p1)
error_letter

#### Go on


In [4]:
# translate the hidden and observed sequences into numeric list
hidden_ = string_to_numbers(hidden_sequence, mapping=map_alphabet_to_numbers())
observed_ = string_to_numbers(observed_sequence, mapping=map_alphabet_to_numbers())

In [5]:
B_start = np.zeros((27, 27)) + 1 / 26
B_start[:, -1] = np.zeros(27)
B_start[-1, :] = np.zeros(27)
B_start[-1, -1] = 1

emission = Baum_Welch(
    A=p.normalized_matrix,
    B_start=B_start,
    pi=p.normalized_matrix[-1, :],
    observed=observed_,
    maxIter=100,
)

In [ ]:
emission.sum(axis=1)

In [6]:
# go back to letters to see the results
mapping = find_mapping(emission.argmax(axis=1))
emission_reconstruction = numbers_to_string(observed_sequence, invert_mapping(mapping))
print(emission_reconstruction)

yd yqdmawl iq sqqmd to xm zdmtql mzcv uzqumaqis fdqqqy qzsv ail qhq qiqq yw jrudcm uwr szayw it aii dmzmwrs frqdzmabil uw dusiul dzxtimtimd qhat luz vuvq qo bm tdqdsh or dfawish qu qddmrsqadr iw iqull whl fozlr yoz mvqw bothmr whqd uwd hof mqsh qu qif id a cqmstiud tvuq vad bqqd vqxywy bysitord qu qqrufq jud as iuwy as zmuflq havq xqmd qduvmllidg uroqdd qvq cuwtydmdq uzqdirq qhqid uww cuzdqdl yt smqqs mvmd qzdofmuwd row t hdof qvq uwdwqr assurdydg qo wmw foiiydg bl yoqguv id syy qz cuqdtdiqd bdytayw adr qvq zd fvmrq ud qodq bysyquds hwof bzq muy xm dmlzcqudt tu acgdowimdgq gdutqiqims muy quhm zz murm thaw halj yuzr wuyqfqrdod d ywsomq qqrofquws adm dqmfil divyrqr od tyzziwg iw rmsquzdadqd tor myumzim oj dmszowrmdqd id gqdqawl told tvq fuildtqd qhml tlfysaliy qyzfqr aimosq thq dumm us tvm zs iw tvq zg whmrm aw ofqyowal sqrvism sharyq ut aboqt yd zdzalil idciqdqd suid tvql lqjq a gdutzitl qvq tigqrm yw dzuid wvmrm sqdbisq yd ottqd ywclqdqd id dqdquzdadq xilis xqq ridmrs saw iqavq oztiuwa

Now let us proceed to apply the Viterbi algorithm to obtain the most likely reconstruction and compare how it performs with the one obtained just by using the emissions.


In [7]:
f0, f = compute_f_log(A=p.normalized_matrix, B=emission, observed=observed_)
pmax, phi = Viterbi_log(f0, f)
reconstruction = reconstruct(pmax, phi)
reconstruction = reconstruction.astype(int)
viterbi_reconstruction = convert_numbers_to_letters(reconstruction)

/Users/michelealessi/Desktop/HMM-for-text-decryption/src/HMM_functions.py:160: RuntimeWarning: divide by zero encountered in log
  tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])
/Users/michelealessi/Desktop/HMM-for-text-decryption/src/HMM_functions.py:169: RuntimeWarning: divide by zero encountered in log
  tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])


In [8]:
print(
    "emission accuracy:",
    np.mean(np.array(list(emission_reconstruction)) == np.array(list(hidden_sequence))),
)

print(
    "Viterbi accuracy:",
    np.mean(np.array(list(viterbi_reconstruction)) == np.array(list(hidden_sequence))),
)

emission accuracy: 0.48385236447520186
Viterbi accuracy: 0.8125720876585929


Qui abbiamo accuracy 80%, bene.


In [9]:
print("True sequence:\n", hidden_sequence)
print("Reconstruction from emission:\n", emission_reconstruction)
print("Reconstruction using Viterbi:\n", viterbi_reconstruction)

True sequence:
 in germany it seems to be pretty much automatic pretty much all the time in france and spain it all depends presumably on social subtleties that you have to be french or spanish to understand in italy why would you even bother when and how much to tip is a question that has been vexing visitors to europe for as long as people have been travelling around the continent outside their own country it seems even europeans don t know the answer according to new polling by yougov in six eu countries britain and the us where as most visitors know but may be reluctant to acknowledge gratuities may make up more than half your waitperson s income europeans are deeply divided on tipping in restaurants for example of respondents in germany told the pollster they typically tipped almost the same as the us in the uk where an optional service charge of about is usually included said they left a gratuity the figure in spain where service is often included in restaurant bills but diners c